In [51]:
import tokenizer
import pickle
import pandas as pd
import numpy as np
import glob
import os
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
import json
import sparse_matrix_functions
import scipy.sparse as scs

In [2]:
path = '/Users/ale/Dropbox (Yadlin Family)/galvanize/capstone/*.txt'

In [3]:
with open("gene_dictionary_final.pickle", "rb") as dict_gene:
        gene_dict = pickle.load(dict_gene)

In [4]:
gene_keys = set(gene_dict.keys())
gene_values = set(gene_dict.values())

In [5]:
gene_dict['her2']

'#erbb2#'

In [6]:
with open("drug_dictionary_final.pickle", "rb") as dict_drug:
        drug_dict = pickle.load(dict_drug)

In [7]:
drug_keys = set(drug_dict.keys())
drug_values = set(drug_dict.values())

In [8]:
drug_dict['trastuzumab']

'#trastuzumab#'

In [9]:
with open("greek_alphabet.pickle", "rb") as dict_greek:
        greek_dict = pickle.load(dict_greek)

In [10]:
import data_frame_creator

In [11]:
test=data_frame_creator.create_data_frame(path, short_list=True)

In [12]:
test.head()

,files,sentences,#abcg2#,#acadl#,#ackr3#,#acpp#,#ado#,#afuresertib#,#agfg1#,#aicda#,...,#voxtalisib#,#wdtc1#,#wnt5a#,#xs#,#zbp1#,#zbtb8os#,#zfyve9#,#znf746#,resist,sensit
0,guidelines.txt,Something that moves progressivley First thing...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test.txt,13 Deregulation of the EGFR PI3K PTEN Akt mTO...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,test.txt,ABSTRACT The EGFR PI3K PTEN Akt mTORC1 GSK-3 p...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,test.txt,The expression of this pathway is frequently a...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,test.txt,"In some breast cancer cases, mutations at cert...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test.shape

(1428, 408)

In [14]:
doc_names, orig_sentences = data_frame_creator.create_corpus()

In [16]:
#orig_sentences

In [ ]:
data_frame_creator.process_data(orig_sentences)

In [52]:
vocab_matrix=data_frame_creator.open_pickle('vocab_matrix.pickle')

In [53]:
vocab_matrix

<2893x406 sparse matrix of type '<class 'numpy.int64'>'
	with 2941 stored elements in Compressed Sparse Row format>

In [19]:
'vocab_matrix.pickle'[-7:]

'.pickle'

In [47]:
vocab_columns=data_frame_creator.open_pickle('vocabulary.pickle')

In [ ]:
vocab_columns=np.array(vocab_columns)

In [ ]:
vocab_columns

In [49]:
col_idx=sparse_matrix_functions.column_indexing(vocab_columns)

In [50]:
col_idx

{'#abcg2#': 0,
 '#acadl#': 1,
 '#ackr3#': 2,
 '#acpp#': 3,
 '#ado#': 4,
 '#afuresertib#': 5,
 '#agfg1#': 6,
 '#aicda#': 7,
 '#airn#': 8,
 '#akt inhibitor gsk2141795#': 9,
 '#akt inhibitor mk2206#': 10,
 '#akt1#': 11,
 '#alb#': 12,
 '#alisertib#': 13,
 '#alpelisib#': 14,
 '#alpi#': 15,
 '#alyref#': 16,
 '#amh#': 17,
 '#anastrozole#': 18,
 '#anpep#': 19,
 '#ar#': 20,
 '#arc#': 21,
 '#areg#': 22,
 '#arhgef7#': 23,
 '#armc9#': 24,
 '#atg12#': 25,
 '#atm#': 26,
 '#atr#': 27,
 '#bank1#': 28,
 '#bax#': 29,
 '#bbc3#': 30,
 '#bcl2#': 31,
 '#bcs1l#': 32,
 '#bevacizumab#': 33,
 '#bid#': 34,
 '#binimetinib#': 35,
 '#birc5#': 36,
 '#bivm#': 37,
 '#bmi1#': 38,
 '#bmp7#': 39,
 '#bpifa4p#': 40,
 '#braf#': 41,
 '#brca1#': 42,
 '#brca2#': 43,
 '#brd2#': 44,
 '#brip1#': 45,
 '#broxuridine#': 46,
 '#buparlisib#': 47,
 '#c19orf66#': 48,
 '#c1qbp#': 49,
 '#ca1#': 50,
 '#capecitabine#': 51,
 '#carbendazim#': 52,
 '#carboplatin#': 53,
 '#cat#': 54,
 '#ccnd1#': 55,
 '#ccng2#': 56,
 '#cct#': 57,
 '#cd24#': 58,


In [44]:
df =data_frame_creator.from_papers_to_panda(path=path,
                      doc_reader=tokenizer.tokenize_many_docs,
                      tokenizer=tokenizer.tokenize,
                      short_list=True,
                      min_df=0.0000001,
                      mat_out='vocab_matrix',
                      vocab_out='vocabulary')

Data has been processed and saved


In [46]:
len(df)

1428

In [60]:
non_empty_rows = sparse_matrix_functions.non_zero_rows(vocab_matrix)

In [61]:
sentences = [orig_sentences[i] for i in non_empty_rows]

In [62]:
files = [doc_names[i] for i in non_empty_rows]

In [64]:
vocab_mat_clean=sparse_matrix_functions.remove_zero_rows(vocab_matrix)

In [66]:
vocab_mat_clean

<1428x406 sparse matrix of type '<class 'numpy.int64'>'
	with 2941 stored elements in Compressed Sparse Row format>

In [74]:
print(vocab_mat_clean)

  (0, 202)	2
  (0, 184)	1
  (1, 113)	1
  (1, 277)	2
  (1, 298)	1
  (1, 11)	1
  (1, 257)	1
  (1, 68)	1
  (1, 404)	1
  (1, 231)	1
  (2, 113)	1
  (2, 277)	1
  (2, 298)	1
  (2, 11)	1
  (2, 404)	1
  (3, 277)	1
  (3, 298)	1
  (3, 119)	1
  (3, 42)	1
  (3, 43)	1
  (3, 381)	1
  (4, 277)	1
  (5, 404)	1
  (5, 119)	1
  (6, 404)	1
  :	:
  (1411, 150)	1
  (1411, 117)	1
  (1412, 105)	1
  (1413, 337)	1
  (1414, 219)	1
  (1414, 176)	1
  (1414, 153)	1
  (1414, 337)	1
  (1415, 219)	1
  (1415, 337)	1
  (1416, 17)	1
  (1417, 360)	1
  (1418, 225)	1
  (1419, 381)	1
  (1420, 44)	1
  (1421, 105)	1
  (1422, 381)	1
  (1422, 219)	1
  (1422, 337)	1
  (1423, 293)	1
  (1424, 144)	1
  (1424, 337)	1
  (1425, 337)	1
  (1426, 16)	1
  (1427, 239)	1


In [76]:
vocab_mat_clean.shape

(1428, 406)

In [78]:
for i in vocab_mat_clean.shape[0]:
    print(i)

TypeError: 'int' object is not iterable

In [80]:
a = scs.csc_matrix(vocab_mat_clean)

In [82]:
print(a)

  (306, 0)	1
  (308, 0)	1
  (445, 0)	1
  (899, 0)	1
  (293, 1)	1
  (297, 1)	1
  (299, 1)	1
  (494, 2)	1
  (134, 3)	1
  (179, 3)	1
  (180, 3)	1
  (181, 3)	1
  (442, 3)	1
  (609, 3)	3
  (832, 3)	1
  (887, 3)	1
  (894, 3)	1
  (960, 3)	1
  (1392, 3)	1
  (561, 4)	1
  (607, 5)	1
  (608, 5)	1
  (230, 6)	1
  (232, 6)	1
  (343, 7)	1
  :	:
  (579, 405)	1
  (580, 405)	1
  (581, 405)	1
  (599, 405)	1
  (619, 405)	1
  (623, 405)	1
  (625, 405)	1
  (662, 405)	1
  (666, 405)	1
  (670, 405)	1
  (671, 405)	1
  (681, 405)	1
  (683, 405)	1
  (684, 405)	1
  (786, 405)	1
  (788, 405)	1
  (887, 405)	1
  (889, 405)	1
  (891, 405)	1
  (927, 405)	1
  (950, 405)	1
  (1021, 405)	1
  (1026, 405)	1
  (1033, 405)	1
  (1091, 405)	1


In [92]:
a.shape[1]

406

In [100]:
b = vocab_mat_clean.nonzero()[0]

In [101]:
c = vocab_mat_clean.nonzero()[1]

In [109]:
rows=[]
for i in b:
    rows.append(i)

In [111]:
len(rows)

2941

In [112]:
cols=[]
for i in b:
    cols.append(i)

In [113]:
len(cols)

2941

In [126]:
d=[]
for i in range(len(cols)):
    d.append((rows[i],cols[i]))

In [127]:
for i in

[(0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (3, 3),
 (3, 3),
 (3, 3),
 (3, 3),
 (3, 3),
 (3, 3),
 (4, 4),
 (5, 5),
 (5, 5),
 (6, 6),
 (6, 6),
 (6, 6),
 (7, 7),
 (7, 7),
 (7, 7),
 (7, 7),
 (8, 8),
 (8, 8),
 (8, 8),
 (8, 8),
 (8, 8),
 (9, 9),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (11, 11),
 (11, 11),
 (12, 12),
 (12, 12),
 (13, 13),
 (13, 13),
 (14, 14),
 (14, 14),
 (15, 15),
 (16, 16),
 (17, 17),
 (17, 17),
 (17, 17),
 (18, 18),
 (19, 19),
 (20, 20),
 (20, 20),
 (21, 21),
 (21, 21),
 (21, 21),
 (21, 21),
 (22, 22),
 (22, 22),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (23, 23),
 (24, 24),
 (24, 24),
 (25, 25),
 (26, 26),
 (26, 26),
 (27, 27),
 (27, 27),
 (28, 28),
 (29, 29),
 (29, 29),
 (30, 30),
 (30, 30),
 (31, 31),
 (32, 32),
 (33, 33),
 (33, 3

In [117]:
print(d)